In [ ]:
xmlNs = {"cmmn": "http://www.omg.org/spec/CMMN/20151109/MODEL"}
xmlPath = "/Users/wvw/Dropbox/research/projects/Montfort PM/cmmn/cap-all.cmmn"

from rdflib import Namespace
CM = Namespace("http://rdf.org/cmmn#")
ST = Namespace("http://rdf.org/state#")
modelNs = Namespace("http://ontario.org/qbp/cap#")
ttlPath = 'ttl/cap0.ttl'

# Parse

In [27]:
import xml.etree.ElementTree as ET
import pprint

def parse(path, ns):
    tree = ET.parse(path)
    for prefix, uri in ns.items():
        ET.register_namespace(prefix, uri)
    root = tree.getroot()
    
    itemObjs = {}
    planItems = root.findall(".//cmmn:planItem", namespaces=ns)

    for planItem in planItems:
        defRef = planItem.attrib['definitionRef']
        
        itemObj = {
            'id': planItem.attrib['id'],
            'sentries': {
                'entry': [],
                'exit': []
            }
        }
        itemObjs[defRef] = itemObj
        
        reqRules = planItem.findall(".//cmmn:itemControl/cmmn:requiredRule", namespaces=ns)
        repRules = planItem.findall(".//cmmn:itemControl/cmmn:repetitionRule", namespaces=ns)
        mandatory = len(reqRules) > 0; repetition = len(repRules) > 0
        itemObj['mandatory'] = mandatory
        itemObj['repetition'] = repetition
        
        label = None; typ = None
        if defRef.startswith("Task"):
            labelNode = root.findall(f".//cmmn:task[@id='{defRef}']", namespaces=ns)[0]
            label = labelNode.attrib['name']
            typ = 'Task'
        elif defRef.startswith("Stage"):
            stage = root.findall(f".//cmmn:stage[@id='{defRef}']", namespaces=ns)[0]
            label = stage.attrib['name']
            itemObj['children'] = []
            for childPlanItem in stage.findall(".//cmmn:planItem", namespaces=ns):
                itemObj['children'].append(childPlanItem.attrib['definitionRef'])
            typ = 'Stage'
        elif defRef.startswith("Milestone"):
            milestone = root.findall(f".//cmmn:milestone[@id='{defRef}']", namespaces=ns)[0]
            label = milestone.attrib['name']
            typ = 'Milestone'
        else:
            continue
            
        itemObj['label'] = label
        itemObj['type'] = typ
        
        itemObj['states'] = [ ( 'Inactive', 'init' ) ]
            
        # print(">", label, ("(mandatory)" if mandatory else ""))
        
        for entryCrit in planItem.findall("cmmn:entryCriterion", namespaces=ns): 
            # print("- sentry")              
            sentry = root.findall(f".//cmmn:sentry[@id='{entryCrit.attrib['sentryRef']}']", namespaces=ns)[0]
            
            sentryObj = {
                'id': entryCrit.attrib['id'],
                'ref': sentry.attrib['id'],
                'items': [],
                'conditions': []
            }
            itemObj['sentries']['entry'].append(sentryObj)
            
            itemParts = sentry.findall(".//cmmn:planItemOnPart", namespaces=ns)
            for itemPart in itemParts:
                sentryItemObj = { 'id': itemPart.attrib['id'] }
                sourcePlanItem = root.findall(f".//cmmn:planItem[@id='{itemPart.attrib['sourceRef']}']", namespaces=ns)[0]
                sourceDefRef = sourcePlanItem.attrib['definitionRef']
                sentryItemObj['source'] = sourceDefRef
                # print("source:", sourceDefRef)
                
                events = itemPart.findall(".//cmmn:standardEvent", namespaces=ns)
                if len(events) > 0:
                    eventLabel = events[0].text
                    sentryItemObj['event'] = eventLabel
                    # print("event:", eventLabel)
            
                sentryObj['items'].append(sentryItemObj)
            
            associations = root.findall(f".//cmmn:association[@sourceRef='{entryCrit.attrib['id']}']", namespaces=ns)
            for association in associations:
                condItemObj = { 'id': association.attrib['targetRef'] }
                textAnnotations = root.findall(f".//cmmn:textAnnotation[@id='{association.attrib['targetRef']}']", namespaces=ns)
                if len(textAnnotations) > 0:
                    textAnnotation = textAnnotations[0].findall(".//cmmn:text", namespaces=ns)[0].text
                    # print("condition:", textAnnotation)
                    condItemObj['text'] = textAnnotation
                    sentryObj['conditions'].append(condItemObj)
                    
        # print("")
    
    return itemObjs

In [28]:
itemObjs = parse(xmlPath, xmlNs)
pprint.pprint(itemObjs)

{'Milestone_0gl7st9': {'id': 'PlanItem_1i1l7uj',
                       'label': 'hospital admission',
                       'mandatory': False,
                       'repetition': False,
                       'sentries': {'entry': [], 'exit': []},
                       'states': [('Inactive', 'init')],
                       'type': 'Milestone'},
 'Stage_1sveclw': {'children': ['Task_0kvjrmn',
                                'Task_0hsxzhz',
                                'Task_0rwjaka',
                                'Task_0akqm8t',
                                'Task_022k641'],
                   'id': 'PlanItem_000hrg1',
                   'label': '',
                   'mandatory': False,
                   'repetition': False,
                   'sentries': {'entry': [{'conditions': [],
                                           'id': 'EntryCriterion_0nvb7u0',
                                           'items': [{'event': 'occur',
                                         

# Convert

In [29]:
from rdflib import Literal, Graph, BNode, RDF, RDFS, XSD, URIRef
from rdflib.collection import Collection
from util import rdf_coll
from convert_base import str_to_uri

def convert(itemObjs, modelNs, dest):
    g = Graph()

    for defRef, itemObj in itemObjs.items():
        planItemUri = str_to_uri(defRef, modelNs)

        if itemObj['label'].strip() != "":
            g.add((planItemUri, RDFS['label'], Literal(itemObj['label'])))
        
        g.add((planItemUri, CM['isMandatory'], Literal(itemObj['mandatory'], datatype=XSD['boolean'])))
        g.add((planItemUri, CM['hasRepetition'], Literal(itemObj['repetition'], datatype=XSD['boolean'])))

        g.add((planItemUri, RDF['type'], CM['PlanItem']))    
        g.add((planItemUri, RDF['type'], CM[itemObj['type']]))
        if itemObj['type'] == 'Stage':
            for child in itemObj['children']:
                g.add((planItemUri, CM['hasChild'], str_to_uri(child, modelNs)))
        
        curState = itemObj['states'][-1]
        g.add((planItemUri, ST['in'], rdf_coll(g, ST[curState[0]], Literal(curState[1]))))
                
        for sentry in itemObj['sentries']['entry']:
            sentryUri = str_to_uri(sentry['id'], modelNs)
            g.add((planItemUri, CM['hasSentry'], sentryUri))
            g.add((sentryUri, RDF['type'], CM['Sentry']))
            g.add((sentryUri, RDF['type'], CM['EntrySentry']))
            
            for item in sentry['items']:
                planItemPartUri = str_to_uri(item['id'], modelNs)
                g.add((sentryUri, CM['hasPlanItemPart'], planItemPartUri))
                
                sourceUri = str_to_uri(item['source'], modelNs)
                g.add((planItemPartUri, CM['hasSource'], sourceUri))
                
                eventLabel = item['event']
                g.add((planItemPartUri, CM['hasEvent'], Literal(eventLabel)))
                
            for condition in sentry['conditions']:
                conditionUri = str_to_uri(condition['id'], modelNs)
                g.add((sentryUri, CM['hasCondition'], conditionUri))
                
                g.add((conditionUri, RDFS['comment'], Literal(condition['text'])))
                
    g.serialize(format="n3", destination=dest)

In [30]:
convert(itemObjs, modelNs, "ttl/cap/model.ttl")

# Reason

## Steps in Python

In [6]:
from util import run
import re

def step(itemObjs, modelNs, dataPath, nr):
    target = dataPath[0:dataPath.index(".")] + str(nr) + dataPath[dataPath.index("."):]
        
    convert(itemObjs, modelNs, target)
    out = reason(target)
    update(itemObjs, out, modelNs)

    for defRef, itemObj in itemObjs.items():
        print(itemObj['label'], f"({defRef})", itemObj['states'])

def reason(dataPath):
    out = run(['eye', 'n3/state.n3', 'n3/workflow.n3', dataPath, '--nope', '--pass-only-new'])
    return out

def update(itemObjs, out, modelNs):    
    nsStr = modelNs.__str__()
    for defRef, state, reason, id in re.findall(f"<{nsStr}(.*?)> state:in \(\s*state:(.*?) \"(.*?)\" (\d+)\s*\)", out):
        itemObjs[defRef]['states'].append((state, reason, id))

In [7]:
# initialize

import copy
itemObjs2 = copy.deepcopy(itemObjs)

step(itemObjs2, modelNs, ttlPath, 0)

time: 0.12709850003011525
x-ray (Task_0u6b4ee) [('Inactive', 'init')]
antibiotics (Task_10t4i0s) [('Inactive', 'init')]
 (Stage_1sveclw) [('Inactive', 'init')]
hospital admission (Milestone_0gl7st9) [('Inactive', 'init')]
urine antigen testing (Task_1dy8o18) [('Inactive', 'init')]
PCR for legionella (Task_1vk3nm4) [('Inactive', 'init')]
Blood cultures (Task_1hoczvg) [('Inactive', 'init')]
sputum culture (Task_0kvjrmn) [('Inactive', 'init')]
gram staining (Task_0hsxzhz) [('Inactive', 'init')]
Blood gases, lactate, liver enzyme and liver function tests (Task_0rwjaka) [('Inactive', 'init')]
routine bloodwork: CBC, electrolytes, renal function (Task_0akqm8t) [('Inactive', 'init')]
oxygen saturation (Task_022k641) [('Inactive', 'init')]


In [8]:
# first observation
states = itemObjs2['Milestone_0gl7st9']['states']
states.append(('Completed', 'observation', len(states)))

In [9]:
step(itemObjs2, modelNs, ttlPath, 1)

time: 0.05605991603806615
x-ray (Task_0u6b4ee) [('Inactive', 'init')]
antibiotics (Task_10t4i0s) [('Inactive', 'init')]
 (Stage_1sveclw) [('Inactive', 'init')]
hospital admission (Milestone_0gl7st9) [('Inactive', 'init'), ('Completed', 'observation', 1)]
urine antigen testing (Task_1dy8o18) [('Inactive', 'init')]
PCR for legionella (Task_1vk3nm4) [('Inactive', 'init')]
Blood cultures (Task_1hoczvg) [('Inactive', 'init')]
sputum culture (Task_0kvjrmn) [('Inactive', 'init')]
gram staining (Task_0hsxzhz) [('Inactive', 'init')]
Blood gases, lactate, liver enzyme and liver function tests (Task_0rwjaka) [('Inactive', 'init')]
routine bloodwork: CBC, electrolytes, renal function (Task_0akqm8t) [('Inactive', 'init')]
oxygen saturation (Task_022k641) [('Inactive', 'init')]


## Steps in N3

### log:conclusion

In [10]:
from util import run 

out = run(['eye', 'n3/concl/state.n3', 'n3/concl/allinone.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

time: 0.046085541136562824
error: eye --quiet n3/concl/state.n3 n3/concl/allinone.n3 --nope --pass-only-new
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 29 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/concl/state.n3 SC=4
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/concl/allinone.n3 SC=6
networking 5 [msec cputime] 5 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/cap.ttl ** ERROR ** error(existence_error(source_sink,/Users/wvw/git/pm/decl_cig/cmmn3/n3/cap.ttl),context(system:open/4,No such file or directory)) **
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/cap.ttl ** ERROR ** error(existence_error(source_sink,/Users/wvw/git/pm/decl_cig/cmmn3/n3/cap.ttl),context(system:open/4,No such file or directory)) **
reasoning 0 [msec cputime] 0 [msec walltime]
2026-01-30T14:04:23.497Z in=10 out=0 ent=0 step=0 brake=2 inf=78131 sec=0.030 inf/sec=2604367


# Processed by EYE v11.23.2 (2025-12-09)
# eye --quiet n3/concl/state.n3 

In [11]:
from util import run 

out = run(['eye', 'n3/concl/test.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

out = run(['npx', 'eyeling', 'n3/concl/test.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

time: 0.039390290854498744
error: eye --quiet n3/concl/test.n3 --nope --pass-only-new
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 28 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/concl/test.n3 ** ERROR ** gre ** error(existence_error(source_sink,/Users/wvw/git/pm/decl_cig/cmmn3/n3/concl/test.n3),context(system:open/4,No such file or directory))

# Processed by EYE v11.23.2 (2025-12-09)
# eye --quiet n3/concl/test.n3 --nope --pass-only-new


time: 0.5804086250718683
error: (node:83387) ExperimentalWarning: CommonJS module /Users/wvw/.nvm/versions/node/v23.2.0/lib/node_modules/npm/node_modules/debug/src/node.js is loading ES Module /Users/wvw/.nvm/versions/node/v23.2.0/lib/node_modules/npm/node_modules/supports-color/index.js using require().
Support for loading ES Module in require() is an experimental feature and might change at any time
(Use `node --trace-warnings ...` to show where the warning was created)
Error reading file "n3/conc

### N3 loop

In [31]:
from util import run 

out = run(['eye', 'ttl/cap/model.ttl', 'ttl/cap/obs.ttl', 'n3/graph.n3', 'n3/state.n3', 'n3/workflow.n3', 'n3/run.n3', '--nope', '--pass-only-new'], printerr=False)
print(out)

time: 0.12790687498636544
# Processed by EYE v11.23.2 (2025-12-09)
# eye --quiet ttl/cap/model.ttl ttl/cap/obs.ttl n3/graph.n3 n3/state.n3 n3/workflow.n3 n3/run.n3 --nope --pass-only-new

@prefix cap: <http://ontario.org/qbp/cap#>.
@prefix ns2: <http://rdf.org/state#>.
@prefix re: <http://rdf.org/cmmn/reason#>.
@prefix er: <http://rdf.org/cmmn/error#>.

("\"\"" cap:Stage_1sveclw) ns2:all ((ns2:Inactive ("init")) (ns2:Ready (re:notInStage)) (ns2:Active (re:sentryFired)) (ns2:Completed (re:childrenDoneOrOptional))).
("antibiotics" cap:Task_10t4i0s) ns2:all ((ns2:Inactive ("init")) (ns2:Ready (re:notInStage)) (ns2:Completed (re:observation er:readyToCompleted))).
("urine antigen testing" cap:Task_1dy8o18) ns2:all ((ns2:Inactive ("init")) (ns2:Ready (re:notInStage er:mandatoryNotDone))).
("Blood cultures" cap:Task_1hoczvg) ns2:all ((ns2:Inactive ("init")) (ns2:Ready (re:notInStage er:mandatoryNotDone))).
("PCR for legionella" cap:Task_1vk3nm4) ns2:all ((ns2:Inactive ("init")) (ns2:Ready (r

# Show

In [32]:
from rdflib import Literal, Graph, BNode, RDF, RDFS, XSD, URIRef
from rdflib.collection import Collection

g = Graph()
g.parse(data=out)

<Graph identifier=N294a5781422345cbaaa84591dec1a9ed (<class 'rdflib.graph.Graph'>)>

In [33]:
from rdflib.namespace import split_uri

errors = {}
finals = {}

for s, _, o in g.triples( ( None, ST['all'], None ) ):
    id = Collection(g, s)
    label, item = id
    _, item = split_uri(item)
    id = itemObjs[item]['id']

    states = [ state for state in Collection(g, o) ]
    for idx, state in enumerate(states):
        typ, dnode = Collection(g, state)
        _, typ = split_uri(typ)
        
        if idx == len(states) - 1:
            finals[item] = id, typ
        
        desc = Collection(g, dnode)
        if len(desc) > 1:
            reason, error = desc
            _, error = split_uri(error)
            errors[item] = id, error, typ

In [34]:
errors

{'Task_10t4i0s': ('PlanItem_1i0a7li', 'readyToCompleted', 'Completed'),
 'Task_1dy8o18': ('PlanItem_0xchgvg', 'mandatoryNotDone', 'Ready'),
 'Task_1hoczvg': ('PlanItem_0ntpscm', 'mandatoryNotDone', 'Ready'),
 'Task_022k641': ('PlanItem_1s8zhhx', 'inactiveToCompleted', 'Completed'),
 'Task_0akqm8t': ('PlanItem_10zx5c3',
  'nonRepetitiveMultipleCompleted',
  'Completed'),
 'Task_0u6b4ee': ('PlanItem_08x0xdn', 'mandatoryNotDone', 'Active')}

In [35]:
showStates = []
extraMarkers = []

for item, ( id, state ) in finals.items():
    showStates.append(f"showState('{id}', '{state.lower()}', canvas, overlays);")
    
for item, ( id, error, itemState )  in errors.items():
    itemObj = itemObjs[item]
    match (error):
        case 'readyToCompleted':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
            for sentry in itemObj['sentries']['entry']:
                showStates.append(f"showState('{sentry['id']}', 'sentryViolated', canvas, overlays);")
                
        case 'inactiveToCompleted':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
        
        case 'mandatoryNotDone':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
            showStates.append(f"showState('{id}', 'firstSymbolViolated', canvas, overlays);")
            
        case 'nonRepetitiveMultipleCompleted':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
            clsName = 'secondSymbolViolated' if (itemObj['mandatory']) else 'firstSymbolViolated'
            showStates.append(f"showState('{id}', '{clsName}', canvas, overlays);")
            extraMarkers.append(f"'{id}': {{ 'isRepeatable': true }}")
            
showStatesJs = "\n".join(showStates)
print(showStatesJs)
extraMarkersJs = ",".join(extraMarkers)
extraMarkersJs = f"window.extraMarkers = {{ { extraMarkersJs } }}"
print(extraMarkersJs)

# from IPython.display import display, HTML
# display(HTML(html))

with open("viewer/viewer-or.html", 'r') as fh:
    html = fh.read()
    
with open("viewer/viewer.html", 'w') as fh:
    html = html.replace("<extraMarkersPlaceholder>", extraMarkersJs)
    html = html.replace("<showStatesPlaceholder>", showStatesJs)
    fh.write(html)

showState('PlanItem_000hrg1', 'completed', canvas, overlays);
showState('PlanItem_1i0a7li', 'completed', canvas, overlays);
showState('PlanItem_0xchgvg', 'ready', canvas, overlays);
showState('PlanItem_0ntpscm', 'ready', canvas, overlays);
showState('PlanItem_1hc9qyp', 'ready', canvas, overlays);
showState('PlanItem_1i1l7uj', 'completed', canvas, overlays);
showState('PlanItem_1s8zhhx', 'completed', canvas, overlays);
showState('PlanItem_10zx5c3', 'completed', canvas, overlays);
showState('PlanItem_04z6f16', 'ready', canvas, overlays);
showState('PlanItem_1bjqkao', 'ready', canvas, overlays);
showState('PlanItem_08x0xdn', 'active', canvas, overlays);
showState('PlanItem_08e8toi', 'ready', canvas, overlays);
showState('PlanItem_1i0a7li', 'error', canvas, overlays);
showState('EntryCriterion_0u33j0f', 'sentryViolated', canvas, overlays);
showState('EntryCriterion_01cmj7b', 'sentryViolated', canvas, overlays);
showState('PlanItem_0xchgvg', 'error', canvas, overlays);
showState('PlanItem_0